In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
# Load libraries
import pandas as pd
import numpy as np
#from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
#from sklearn.metrics import classification_report
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing

In [3]:
training = pd.read_csv('../csvs/training.csv')
test = pd.read_csv('../csvs/test.csv')

In [4]:
training.drop(['sepostulo'], 1, inplace = True)
test.drop(['sepostulo'], 1, inplace = True)
areas = training['nombre_area']
resultados = np.array(test['nombre_area'])

In [5]:
training.head()

,edad,estado,nivel,nombre,nombre_area,nombre_zona,sexo,tipo_de_trabajo,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior
0,49.0,1.0,7.0,6.0,2,7,1,1,0,1,0,0,0
1,49.0,2.0,6.0,4.0,2,7,1,1,0,1,0,0,0
2,58.0,1.0,5.0,4.0,2,7,1,1,0,1,0,0,0
3,58.0,2.0,6.0,4.0,2,7,1,1,0,1,0,0,0
4,42.0,1.0,4.0,3.0,2,7,1,1,0,1,0,0,0


In [6]:
test.head()

,edad,estado,nivel,nombre,nombre_area,nombre_zona,sexo,tipo_de_trabajo,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior
0,42.0,2.0,9.0,7.0,30,1,0,1,0,1,0,0,0
1,31.0,2.0,5.0,3.0,30,1,1,1,0,1,0,0,0
2,36.0,2.0,6.0,4.0,30,1,0,1,0,1,0,0,0
3,68.0,2.0,8.0,6.0,170,1,1,1,0,0,0,1,0
4,32.0,1.0,5.0,4.0,170,1,0,1,0,0,0,1,0


In [31]:
resultados

array([ 30,  30,  30, ..., 138, 138, 138])

### Split-out validation dataset
X = np.array(training.drop(['nombre_area'], 1))
Y = np.array(areas)
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

X

Y

seed = 7
scoring = 'accuracy'
model = GaussianNB()
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
msg = "%s: %f (%f)" % ('NB', cv_results.mean(), cv_results.std())
print(msg)

In [7]:
test.fillna(0,inplace=True)

In [8]:
test.head()

,edad,estado,nivel,nombre,nombre_area,nombre_zona,sexo,tipo_de_trabajo,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior
0,42.0,2.0,9.0,7.0,30,1,0,1,0,1,0,0,0
1,31.0,2.0,5.0,3.0,30,1,1,1,0,1,0,0,0
2,36.0,2.0,6.0,4.0,30,1,0,1,0,1,0,0,0
3,68.0,2.0,8.0,6.0,170,1,1,1,0,0,0,1,0
4,32.0,1.0,5.0,4.0,170,1,0,1,0,0,0,1,0


In [33]:
# Make predictions on validation dataset
gnb = GaussianNB()
gnb.fit(training, areas)
predictions = gnb.predict_proba(test)
predictions.shape
indicador = 0
col_names =  ['id', 'sepostulo']
final  = pd.DataFrame(columns = col_names)
for prediction in predictions:
    #msg = "%f: %i" % (prediction, resultados[indicador])
    #msg = "%f: %i" % (prediction[resultados[indicador]], resultados[indicador])
    if(resultados[indicador] >= 186):
        resultado_parcial = [indicador, 0]
    else:
        resultado_parcial = [indicador, prediction[resultados[indicador]]]
    final.loc[len(final)] = resultado_parcial
    indicador += 1

In [34]:
print(indicador)

100000


In [36]:
final.head()

,id,probabilidad
0,0.0,0.0
1,1.0,0.0
2,2.0,0.0
3,3.0,0.0
4,4.0,0.0


In [38]:
final.shape

(100000, 2)

In [50]:
final = final.round({'sepostulo': 1})
final['id'] = final['id'].astype(int)

In [54]:
final

,id,sepostulo
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
5,5,0.0
6,6,0.0
7,7,0.0
8,8,0.0
9,9,0.0


In [55]:
final.to_csv('../csvs/gaussianNB.csv', index = False)